# Tutorium 2

## Mathematisches Modell

**Zielfunktion**

\begin{equation}
	minimiere\ \ Z = \sum^{I}_{i=1} f_{i} \cdot y_{i} + \sum^{I}_{i=1} \sum^{J}_{j=1} c_{ij} \cdot x_{ij}
\end{equation}

**unter den Nebenbedingungen**

\begin{align}
&& \sum^{I}_{i=1} x_{ij} &= d_{j} && \forall j \in J \\[5pt]
&& \sum^{J}_{j=1} x_{ij} &\leq b_{i} \cdot y_{i} && \forall i \in I \\[10pt]
&& x_{ij} &\geq 0 && \forall i \in I, \forall j \in J \\[5pt]
&& y_{i} &\in \left\{ 0, 1 \right\} &&
\end{align}


## Aufgabe 3b)
Lesen Sie aus den csv-Dateien die erforderlichen Daten ein inkl. der Namen der Absatzorte und Standorte.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import csv

### Absatzorte.csv

| Ort       | Bedarf | 
|:----------|--------| 
| Frankfurt | 200    | 
| Berlin    | 250    | 
| München   | 150    | 
| Köln      | 300    | 
| Hamburg   | 100    | 

In [ ]:
d=[]
ao=[]
with open("Absatzorte.csv", encoding="utf-8") as csv_file:
     csv_reader = csv.DictReader(csv_file)
     for row in csv_reader:
          d.append(int(row["Bedarf"]))
          ao.append(row["Ort"])

### Standorte.csv

| Ort        | Kapazität | Fixkosten | 
|:-----------|-----------|-----------| 
| Dresden    | 400       | 60000     | 
| Bremen     | 350       | 60000     | 
| Düsseldorf | 600       | 80000     | 

In [ ]:
b=[]
f=[]
so=[]
with open("Standorte.csv", encoding="utf-8") as csv_file:
     csv_reader = csv.DictReader(csv_file)
     for row in csv_reader:
          b.append(int(row["Kapazität"]))
          f.append(int(row["Fixkosten"]))
          so.append(row["Ort"])

Hinweis: Zum Einlesen von der Distanz und der Fahrtdauer empfiehlt sich ein Einlesen mit `csv.reader()`, da nicht mit den Spaltennamen gearbeitet werden muss.

### Dauer.csv

| Orte       | Frankfurt | Berlin | München | Köln | Hamburg | 
|:-----------|-----------|--------|---------|------|---------| 
| Dresden    | 4         | 2      | 5       | 6    | 5       | 
| Bremen     | 4         | 4      | 8       | 3    | 2       | 
| Düsseldorf | 2         | 6      | 7       | 1    | 5       | 

In [ ]:
dur=[]
with open("Dauer.csv", encoding="utf-8") as csv_file:
     csv_reader = csv.reader(csv_file)
     next(csv_reader)
     for row in csv_reader:
          rowAsInt = [int(item) for item in row[1:]]
          dur.append(rowAsInt)

### Enfernung.csv

| Orte       | Frankfurt | Berlin | München | Köln | Hamburg | 
|:-----------|-----------|--------|---------|------|---------| 
| Dresden    | 460       | 190    | 460     | 570  | 500     | 
| Bremen     | 440       | 400    | 750     | 320  | 130     | 
| Düsseldorf | 230       | 560    | 610     | 40   | 410     | 


In [ ]:
dist=[]
with open("Entfernung.csv", encoding="utf-8") as csv_file:
     csv_reader = csv.reader(csv_file)
     next(csv_reader)
     for row in csv_reader:
          rowAsInt = [int(item) for item in row[1:]]
          dist.append(rowAsInt)

## Aufgabe 3c)
Da Ihnen die Werte für die Transportkosten nicht gegeben sind, berechnen Sie diese aus den vorhandenen Daten Dauer und Entfernung. Gehen Sie von einem km-Kostensatz von 0,75 GE/km und einem Stundenlohn von 45 GE aus.

### Definition der Mengen

In [ ]:
I_max = len(b)
J_max = len(d)

I = range(I_max)
J = range(J_max)

### Berechnung der Kosten

In [ ]:
costPerKm = 0.75
costperH = 45

In [ ]:
c = []
for i in I:
     newRow = []
     for j in J:
          newRow.append(costperH * dur[i][j] + costPerKm * dist[i][j])
     c.append(newRow)

## Aufgabe 3d)
Lassen Sie sich zur Kontrolle jede Zeile der berechneten Kostenmatrix in der Konsole ausgeben und lösen Sie das Problem.

In [ ]:
for row in c:
    print(row)

Initialisierung des Modells:

In [ ]:
m = gp.Model()

Initialisierung der Variablen:

In [ ]:
x = {}
for i in I:
     for j in J:
          x[i,j] = m.addVar(vtype=GRB.CONTINUOUS, name="x_"+str(i)+str(j))

In [ ]:
y = {}
for i in I:
     y[i] = m.addVar(vtype=GRB.BINARY, name="y_"+str(i))

Definition der Zielfunktion

In [ ]:
m.setObjective(gp.quicksum(f[i] * y[i] for i in I) + gp.quicksum(c[i][j] * x[i,j] for j in J for i in I), GRB.MINIMIZE)

Hinzufügen der Nebenbedingungen

In [ ]:
for j in J:
     m.addConstr(gp.quicksum(x[i,j] for i in I) == d[j], "nb_" + str(j))

In [ ]:
for i in I:
     m.addConstr(gp.quicksum(x[i,j] for j in J) <= b[i] * y[i], "kb_" + str(i))

Optimierung:

In [ ]:
m.optimize()

## Aufgabe 3e)
Schreiben Sie in die Konsole als Vorbereitung eines aussagekräftigen Ergebnisses den Satz: "Optimale Standortwahl bei mehreren Betriebsstätten".

In [ ]:
print("\nOptimale Standortwahl bei mehreren Betriebsstätten\n")

## Aufgabe 3f)
Geben Sie den Zielfunktionswert innerhalb eines Satzes an.

In [ ]:
print("Die gesamten Kosten des Transportes betragen", m.getAttr(GRB.Attr.ObjVal), "GE.")

## Aufgabe 3g)
Formulieren Sie anschließend die Standortwahl.

In [ ]:
for i in I:
     if y[i].X > 0:
          print("In", so[i], "wird ein Standort errichtet.")
          print("\tFixkosten:", f[i], "GE")
          print("\tVariable Kosten:", sum([c[i][j] * x[i,j].X for j in J]), "GE")
     else:
          print("In", so[i], "wird KEIN Standort errichtet.")

## Aufgabe 3h)
Des Weiteren sollen die Lieferungen zwischen den Standorten sowie die jeweils zugehörigen Transportkosten in der Konsole beschrieben werden.

In [ ]:
for i in I:
     for j in J:
          if x[i,j].X > 0:
               print("Von", so[i], "werden", x[i,j].X, "ME nach", ao[j], "geliefert, die Transportkosten betragen dabei", c[i][j] * x[i,j].X, "GE.")

## Aufgabe 3i)
Erzeugen Sie eine Textdatei "Ergebnisausgabe.txt" und schreiben Sie in diese Datei die Ausgabe von e) -- h).

In [ ]:
with open("Ergebnisausgabe.txt", "w", encoding="utf-8") as file:
     file.write("Optimale Standortwahl bei mehreren Betriebsstätten\n\n")
     file.write("Die gesamten Kosten des Transportes betragen " + str(m.getAttr(GRB.Attr.ObjVal)) + " GE.\n")
     for i in I:
          if y[i].X > 0:
               file.write("In " + so[i] + " wird ein Standort errichtet!\n")
               file.write("\tFixkosten: " + str(f[i]) + "GE \n")
               file.write("\tVariable Kosten: " + str(sum([c[i][j] * x[i,j].X for j in J])) + " GE\n")
          else:
               file.write("In " + so[i] + " wird KEIN Standort errichtet!\n")
     file.write("\n")
     for i in I:
          for j in J:
               if x[i,j].X > 0:
                    file.write("Von " + so[i] + " werden " + str(x[i,j].X) + " ME nach " + ao[j] +
                    " geliefert, die Transportkosten betragen dabei " + str(c[i][j] * x[i,j].X) + " GE.\n")